In [17]:
# coding: utf-8
import pandas as pd
import os
import numpy as np
import sys
import glob
from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from operator import itemgetter
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cross_decomposition import CCA
from scipy import optimize
from scipy import interpolate
from sklearn import svm
import string
from sklearn.linear_model import LinearRegression
import numpy.ma as ma

In [18]:
date_list=["year","month","day"]

vname_list=['sip_af','area_af','sip_ts','area_ts'] #for Nakata [2019] data
vname_list3=['t2m','u10','v10'] # for atmoshepric data on point
vname_list4=['sip_to','area_to']

pol_list=['CDP','MBP','BaP','SP','VBP','CPP','DaP','DiP','MP','TNBP','RISP','AP','BeP','RONP','ELSP','BISP','BBP']
pol_list2=['CDP','MBP','BaP','SP','VBP','DaP','DiP','MP','TNBP','RISP','AP','BeP','RONP','CPP','ELSP','BISP','BBP']

amse_file=r'polynya_statistics1_softce.csv'
ams2_file=r'polynya_statistics1_softc.csv'
u10file=r'../atmosphere_analysis/u10.npy'
v10file=r'../atmosphere_analysis/v10.npy'
t2mfile=r'../atmosphere_analysis/t2m.npy'


In [2]:
#read nakata data
col = pd.MultiIndex.from_product([vname_list,pol_list], names=('val','pol'))
df1 = pd.read_table(amse_file, sep='\s+',names=col)
for pol in pol_list:
    df1[('area_to',pol)]=df1[('area_af',pol)]+df1[('area_ts',pol)]
    df1.loc[df1[('area_to',pol)]==1998] = 999
for pol in pol_list:
    df1[('sip_to',pol)]=df1[('sip_af',pol)]+df1[('sip_ts',pol)]
    df1.loc[df1[('sip_to',pol)]==1998] = 999
df1.insert(0,'Sensor','AMSR-E')

col = pd.MultiIndex.from_product([vname_list,pol_list], names=('val','pol'))
df2 = pd.read_table(ams2_file, sep='\s+',names=col)
for pol in pol_list:
    df2[('area_to',pol)]=df2[('area_af',pol)]+df2[('area_ts',pol)]
    df2.loc[df2[('area_to',pol)]==1998] = 999
for pol in pol_list:
    df2[('sip_to',pol)]=df2[('sip_af',pol)]+df2[('sip_ts',pol)]
    df2.loc[df2[('sip_to',pol)]==1998] = 999
df2.insert(0,'Sensor','AMSR2')
df3 = pd.concat([df1,df2],axis=0)

# #read era5 data
t2m=np.load(t2mfile)
u10=np.load(u10file)
v10=np.load(v10file)
atmos_p=np.concatenate([t2m,u10,v10],axis=1)
col = pd.MultiIndex.from_product([vname_list3,pol_list2], names=('val','pol'))
df4 = pd.DataFrame(atmos_p,columns=col)


#create date list
date_list=[]
for year in [2003,2004,2005,2006,2007,2008,2009,2010,2012,2013,2014,2015,2016,2017,2018,2019]:
    for month in range(2,12):
        day='1'
        strdt = dt.strptime(str(year) + str(month) + day, '%Y%m%d')
        enddt = strdt + relativedelta(months=1)-timedelta(days=1)
        days_num = (enddt - strdt).days + 1
        date_list.extend([strdt + timedelta(days=x) for x in range(days_num)])
# print(df3.shape,df4.shape,len(date_list))
# # concat datasets with date index
# df=pd.concat([s.reset_index(drop=True) for s in [df3,df4]],axis=1) \
#     .assign(Date=pd.to_datetime(date_list)).set_index('Date')
# df
# df.to_pickle('timeseries.pkl')

print(len(date_list))
print(df3)
#for revise
df=df3.reset_index(drop=True).assign(Date=pd.to_datetime(date_list)).set_index('Date')
df
df.to_pickle('timeseries.pkl')



NameError: name 'pd' is not defined

                     sip_af                                               \
                        CDP  MBP       BaP        SP       VBP       CPP   
Date       Sensor                                                          
2003-03-01 AMSR-E  0.000000  0.0  0.000027  0.001919  0.039662  0.066405   
2003-03-02 AMSR-E  0.000000  0.0  0.003693  0.000000  0.045286  0.054582   
2003-03-03 AMSR-E  0.000000  0.0  0.017979  0.000029  0.005538  0.012464   
2003-03-04 AMSR-E  0.000000  0.0  0.028473  0.002402  0.049790  0.146944   
2003-03-05 AMSR-E  0.000000  0.0  0.168175  0.000708  0.022004  0.182678   
...                     ...  ...       ...       ...       ...       ...   
2019-11-26 AMSR2   0.000017  0.0  0.000353  0.000000  0.002769  0.000000   
2019-11-27 AMSR2   0.000000  0.0  0.000000  0.000000  0.004602  0.000000   
2019-11-28 AMSR2   0.008316  0.0  0.004499  0.017672  0.004626  0.000000   
2019-11-29 AMSR2   0.000000  0.0  0.003134  0.022091  0.039104  0.008917   
2019-11-30 A